In [761]:
import pandas as pd
import numpy as np

In [762]:
HofP=pd.read_json('./HeroesOfPymoli/purchase_data2.json')
HofP.head(10)

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43
5,8,Male,8,"Purgatory, Gem of Regret",2.22,Hainaria90
6,40,Male,148,"Warmonger, Gift of Suffering's End",4.65,Aerithllora36
7,28,Male,27,"Riddle, Tribute of Ended Dreams",3.38,Undirra90
8,18,Male,111,Misery's End,1.79,Eolideu96
9,36,Male,139,"Mercy, Katana of Dismay",4.25,Aesurstilis64


### Player Count

In [763]:
pd.DataFrame(data={'Total Players': [HofP['SN'].nunique()]})

,Total Players
0,74


### Purchasing Analysis (Total)

In [764]:
keys =['Number of Unique Items','Average Price','Number of Purchases','Total Revenue']
unique_items= HofP['Item ID'].nunique()                  
ave_price = HofP['Price'].mean()
total_purchase = HofP['Item ID'].count()
total_revenue = HofP['Price'].sum()
purchase_ana = {'Number of Unique Items': [unique_items],'Average Price': [ave_price],
                'Number of Purchases': [total_purchase],'Total Revenue': [total_revenue]}
purchase_ana_df = pd.DataFrame(data=purchase_ana, columns=keys)
purchase_ana_df['Average Price'] = purchase_ana_df['Average Price'].map('${:,.2f}'.format)
purchase_ana_df['Total Revenue'] = purchase_ana_df['Total Revenue'].map('${:,.2f}'.format)
purchase_ana_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,64,$2.92,78,$228.10


### Gender Demographics

In [765]:
cols=['Percentage of Players',' Total Count ']
genders=[HofP['Gender'].unique()]
gender_grp_counts=pd.DataFrame(HofP.groupby(HofP['Gender'])['SN'].nunique(), 
                               index=genders)
per_plyr=pd.DataFrame(gender_grp_counts['SN']/gender_grp_counts['SN'].sum()*100)
res=per_plyr.set_index(genders).join(gender_grp_counts.set_index(genders),
                                                     lsuffix='0', rsuffix='1')
res['SN0'] = res['SN0'].map('{:,.2f}'.format)
res.rename(columns={'SN0':cols[0],'SN1':cols[1]})

,Percentage of Players,Total Count
Male,81.08,60
Female,17.57,13
Other / Non-Disclosed,1.35,1


### Purchasing Analysis (Gender)

In [766]:
HofP['Normalized Totals']=pd.DataFrame(HofP.groupby(['Gender'])['Price']
                                       .apply(lambda x: (x) / x.count()))
pur_ana=pd.DataFrame(HofP.groupby(['Gender']).agg({'Item ID': 'count',
                                                    'Price': 'mean',
                                                   'Normalized Totals':'sum',
                                                     })).rename(columns={'Item ID':'Purchase Count',
                                                                         'Price':'Average Purchase Price'})
pur_ana['Total Purchase Value']= pur_ana['Purchase Count']*pur_ana['Average Purchase Price']
pur_ana['Total Purchase Value']= pur_ana['Total Purchase Value'].map('${:,.2f}'.format)
pur_ana['Normalized Totals']= pur_ana['Normalized Totals'].map('${:,.2f}'.format)
pur_ana['Average Purchase Price']= pur_ana['Average Purchase Price'].map('${:,.2f}'.format)
pur_ana

,Purchase Count,Average Purchase Price,Normalized Totals,Total Purchase Value
Gender,,,,
Female,13,$3.18,$3.18,$41.38
Male,64,$2.88,$2.88,$184.60
Other / Non-Disclosed,1,$2.12,$2.12,$2.12


In [767]:
HofP1=HofP
HofP1['Percent Count']=HofP['SN']
HofP1['Total Count']=HofP['SN']

def Pyr_count(SN):
    return (len(SN.unique()))
def Perc_Pyr(SN):
    return (len(SN.unique())/HofP['SN'].nunique()*100)

rlabel=[[ "{0}-{1}".format(i+1, i + 4) for i in range(HofP['Age'].min()-1,
                                                       HofP['Age'].max()+4, 4) ]]
rlabel[0][0]='<'+ rlabel[0][0][-2:]
rlabel[0][len(rlabel[0])-1]=rlabel[0][len(rlabel[0])-1][:2]+'+'

a=HofP1.groupby(pd.cut(HofP1['Age'] , 
                    bins=np.arange(HofP1['Age'].min()-1,
                                   HofP1['Age'].max()+8,4),
                      right=True)).agg({'Percent Count':Perc_Pyr,
                                       'Total Count':Pyr_count}
                                      )
a.index=rlabel
a.fillna(0, inplace=True)
a['Total Count']=a['Total Count'].astype(int)
a['Percent Count']=a['Percent Count'].map('{:,.2f}'.format)
a

,Percent Count,Total Count
<10,6.76,5
11-14,4.05,3
15-18,14.86,11
19-22,25.68,19
23-26,28.38,21
27-30,5.41,4
31-34,5.41,4
35-38,6.76,5
39-42,2.70,2
43+,0.00,0


In [768]:
for a, b in HofP.groupby(pd.cut(HofP['Age'] , 
                    bins=np.arange(HofP['Age'].min()-1,
                                   HofP['Age'].max()+8,4), 
                      right=True)):
    print(a,len(b['SN'].unique()))
    #print((b['SN'].unique()))

(6, 10] 5
(10, 14] 3
(14, 18] 11
(18, 22] 19
(22, 26] 21
(26, 30] 4
(30, 34] 4
(34, 38] 5
(38, 42] 2
(42, 46] 0
